# Классификация: IC50 > медианы

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Подготовка 
y = (df["IC50, mM"] > df['IC50, mM'].median()).astype(int)
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение моделей
logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)
y_pred_logit = logit.predict(X_test_scaled)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)

for name, preds, clf, xtest in [("Logistic Regression", y_pred_logit, logit, X_test_scaled),
                                ("Random Forest", y_pred_rf, rf, X_test),
                                ("XGBoost", y_pred_xgb, xgb, X_test),
                                ("LightGBM", y_pred_lgb, lgb, X_test)]:
    print(f"{name}: Accuracy={accuracy_score(y_test, preds):.3f}, Precision={precision_score(y_test, preds):.3f}, Recall={recall_score(y_test, preds):.3f}, F1={f1_score(y_test, preds):.3f}, AUC={roc_auc_score(y_test, clf.predict_proba(xtest)[:, 1]):.3f}")


C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:21:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 399, number of negative: 399
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002167 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18231
[LightGBM] [Info] Number of data points in the train set: 798, number of used features: 170
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

# Классификация: предсказание активности (IC50 > медианы)

В этом разделе выполняются следующие шаги:

1. **Формирование целевой переменной**  
   - Вычисляем медиану IC50 (`df["IC50, mM"].median()`).  
   - Создаём бинарный признак `y`, где  
     - `1` = «IC50 > медианы» (низкая активность),  
     - `0` = «IC50 ≤ медианы» (высокая активность).

2. **Разделение данных и масштабирование**  
   - Стратифицированный сплит на обучающую и тестовую выборки (`train_test_split(..., stratify=y, test_size=0.2)`), чтобы сохранить баланс классов.  
   - Применяем `StandardScaler` к признакам для моделей, чувствительных к масштабу (LogisticRegression).

3. **Обучение моделей**  
   - **LogisticRegression** — базовый линейный классификатор.  
   - **RandomForestClassifier** — ансамбль решающих деревьев.  
   - **XGBClassifier** — градиентный бустинг XGBoost.  
   - **LGBMClassifier** — градиентный бустинг LightGBM.

4. **Подбор гиперпараметров**  
   - Для нелинейных моделей (RF, XGBoost, LightGBM) используется `GridSearchCV` с 5-кратной кросс-валидацией по числу деревьев, глубине и скорости обучения.

5. **Оценка качества**  
   На тестовой выборке получены следующие метрики:

   | Модель               | Accuracy | Precision | Recall | F1    | ROC AUC |
   |----------------------|----------|-----------|--------|-------|---------|
   | Logistic Regression  | 0.700    | 0.682     | 0.750  | 0.714 | 0.759   |
   | Random Forest        | 0.740    | 0.740     | 0.740  | 0.740 | 0.794   |
   | XGBoost              | 0.710    | 0.694     | 0.750  | 0.721 | 0.765   |
   | LightGBM             | 0.705    | 0.703     | 0.710  | 0.706 | 0.775   |

6. **Сравнение моделей и выводы**  
   - **Random Forest** и **LightGBM** демонстрируют наивысшую точность (74.0% и 70.5% соответственно).  
   - **Random Forest** показывает идеальный баланс precision и recall (0.740).  
   - **Logistic Regression** обеспечивает наибольшее recall (0.750) при чуть более низкой precision (0.682).  
   - **XGBoost** сочетает хорошую точность (71.0%) и сбалансированную метрику F1 (0.721).  
   - **Random Forest** лидирует по ROC AUC (0.794), что говорит о лучшем ранжировании вероятностей.

> **Вывод:** для задачи классификации «IC50 > медианы» наилучший баланс метрик достигается с помощью **Random Forest** (Accuracy 0.740, Precision/Recall/F1 = 0.740, ROC AUC 0.794). Модели градиентного бустинга также показывают стабильные результаты и могут конкурировать по эффективности.


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()
